## Face extraction using visual words

---

In this notebook I will use bag-of-visual generated by the [last notebook](https://colab.research.google.com/drive/1J5B1rTAGaAfFelf8P9d4lXzjjH1j_WBr?usp=sharing) and apply multiple models in order to select the best for bounding box. 

---

references:
  - sklearn for multiregression : [models](https://scikit-learn.org/stable/modules/multiclass.html)
  - save and load models : [post](https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/)

In [1]:
!gdown 1O5O20iNpg_aNiNLXc16gIuvUZ3j6S9V0

Downloading...
From: https://drive.google.com/uc?id=1O5O20iNpg_aNiNLXc16gIuvUZ3j6S9V0
To: /content/wider_data_with_visual_words.csv
100% 9.48M/9.48M [00:00<00:00, 221MB/s]


In [2]:
!mkdir models

In [3]:
from sklearn.metrics import classification_report,plot_confusion_matrix,confusion_matrix
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.layers import *
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
from keras.utils.vis_utils import plot_model

from matplotlib import pyplot as plt
from sklearn.svm import LinearSVC 
from sklearn import metrics
import plotly.express as px
import seaborn as sns
from tqdm import tqdm
import pandas as pd
import numpy as np
import pickle
import ast


tqdm.pandas()
sns.set_theme()
MODEL_DIR_NAME = 'models/'

In [33]:
def validation_metrics(y_pred, y_true, show_img=True):
  rmse = np.sqrt(np.square(y_pred - y_true).mean(axis=-1)) * 480
  fig = px.histogram(rmse)
  fig.show()
  return np.mean(rmse)

In [5]:
df = pd.read_csv('wider_data_with_visual_words.csv')
df['visual_words_histogram'] = [np.array(ast.literal_eval(cur_vword)) for cur_vword in df['visual_words_histogram']]
df[['x0', 'y0', 'x1', 'y2']] = df[['x0', 'y0', 'x1', 'y2']] / 480 
df['visual_words_histogram'] = df['visual_words_histogram'].apply(lambda x: x/sum(x))
df.head(3)

,img_location,x0,y0,x1,y2,visual_words_histogram
0,wider_data_resized/0_Parade_marchingband_1_849...,0.438477,0.238267,0.557617,0.345848,"[0.002308136180034622, 0.0005770340450086555, ..."
1,wider_data_resized/0_Parade_Parade_0_904.jpg,0.352539,0.068436,0.609375,0.305168,"[0.007170795306388526, 0.0045632333767926985, ..."
2,wider_data_resized/0_Parade_Parade_0_577.jpg,0.287109,0.189514,0.731445,0.557170,"[0.0023612750885478157, 0.0015741833923652105,..."


In [6]:
df_train, df_test = train_test_split(df, test_size=0.3, random_state=42)

In [7]:
len(df_train), len(df_test)

(4029, 1728)

In [8]:
x_train = np.array(list(df_train['visual_words_histogram'].values))
y_train = np.array(list(df_train[['x0', 'y0', 'x1', 'y2']].values))

x_test = np.array(list(df_test['visual_words_histogram'].values))
y_test = np.array(list(df_test[['x0', 'y0', 'x1', 'y2']].values))

In [9]:
def save_keras_model(model_name, model):
  model.save('models/' + model_name)

## Defining the models for bounding box generation

---

Now we define a model predict the bounding boxes of the image based on bag of visual words. 

So first we can try some Keras NN :

 - Simple dense nn
 - Simple dense nn with skip connections


In order to check the model performance we generate the histogram of RMSE of each example, them we display the mean of the RMSE distribution. The perfect model will have a mean RMSE equal to zero and a histogram concentrated around zero.

In [10]:
from tensorflow.keras.layers import *
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
from keras.utils.vis_utils import plot_model


#### Simple dense NN

---

In [11]:
def simple_dense_model(dropout=0.5,n_neurons=64, num_classes=4, lr=1e-4):

  inputs = Input(shape=x_train[0].shape[0])
  x = Dense(n_neurons, activation="relu")(inputs)
  x = Dense(n_neurons, activation="relu")(x)
  x = Dropout(dropout)(x)
  outputs = Dense(num_classes, activation="sigmoid")(x)

  ret_model = Model(inputs=inputs, outputs=outputs)

  opt = Adam(learning_rate=lr)
  ret_model.compile(opt, "mean_squared_error")
  return ret_model

In [12]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [13]:
simple_dense = simple_dense_model(dropout=0.7,n_neurons=64, lr=1e-4)
simple_dense.fit(x_train,y_train, batch_size=64, epochs=500, 
                validation_split=0.2, shuffle=True, verbose=True,  callbacks=[callback])

Epoch 1/500
51/51 [==============================] - 3s 6ms/step - loss: 0.0508 - val_loss: 0.0495
Epoch 2/500
51/51 [==============================] - 0s 3ms/step - loss: 0.0474 - val_loss: 0.0457
Epoch 3/500
51/51 [==============================] - 0s 3ms/step - loss: 0.0432 - val_loss: 0.0412
Epoch 4/500
51/51 [==============================] - 0s 3ms/step - loss: 0.0387 - val_loss: 0.0368
Epoch 5/500
51/51 [==============================] - 0s 3ms/step - loss: 0.0350 - val_loss: 0.0333
Epoch 6/500
51/51 [==============================] - 0s 3ms/step - loss: 0.0324 - val_loss: 0.0310
Epoch 7/500
51/51 [==============================] - 0s 3ms/step - loss: 0.0311 - val_loss: 0.0297
Epoch 8/500
51/51 [==============================] - 0s 4ms/step - loss: 0.0301 - val_loss: 0.0289
Epoch 9/500
51/51 [==============================] - 0s 3ms/step - loss: 0.0297 - val_loss: 0.0284
Epoch 10/500
51/51 [==============================] - 0s 3ms/step - loss: 0.0294 - val_loss: 0.0281
Epoch 11/

In [14]:
save_keras_model('simple_dense', simple_dense)

INFO:tensorflow:Assets written to: models/simple_dense/assets


In [15]:
pred = simple_dense.predict(x_test)
validation_metrics(pred, y_test)

64.31611109744026

### Dense with skip connections

---

In [16]:
def simple_dense_with_skip_model(dropout=0.5,n_neurons=64, num_classes=4, lr=1e-4):

  inputs = Input(shape=x_train[0].shape[0])

  x0 = Dense(n_neurons, activation="relu")(inputs)
  x = Dense(n_neurons, activation="relu")(x0)
  x0 = Add()([x, x0])

  x = Dense(n_neurons, activation="relu")(x0)
  x = Dense(n_neurons, activation="relu")(x)
  x0 = Add()([x, x0])

  x = Dense(n_neurons, activation="relu")(x0)
  x = Dense(n_neurons, activation="relu")(x)
  x0 = Add()([x, x0])

  x = Dropout(dropout)(x0)

  outputs = Dense(num_classes, activation="sigmoid")(x)

  simple_dense_skip = Model(inputs=inputs, outputs=outputs)

  opt = Adam(learning_rate=lr)
  simple_dense_skip.compile(opt, "mean_squared_error")
  return simple_dense_skip

In [40]:
dense_with_skip = simple_dense_with_skip_model(dropout=0.7,n_neurons=512, lr=1e-4)
dense_with_skip.fit(x_train,y_train, batch_size=128, epochs=500, 
                    validation_split=0.1, shuffle=True, verbose=True,  callbacks=[callback])

Epoch 1/500
29/29 [==============================] - 1s 9ms/step - loss: 0.0406 - val_loss: 0.0294
Epoch 2/500
29/29 [==============================] - 0s 5ms/step - loss: 0.0270 - val_loss: 0.0293
Epoch 3/500
29/29 [==============================] - 0s 5ms/step - loss: 0.0263 - val_loss: 0.0292
Epoch 4/500
29/29 [==============================] - 0s 4ms/step - loss: 0.0261 - val_loss: 0.0292
Epoch 5/500
29/29 [==============================] - 0s 5ms/step - loss: 0.0261 - val_loss: 0.0291
Epoch 6/500
29/29 [==============================] - 0s 5ms/step - loss: 0.0259 - val_loss: 0.0291
Epoch 7/500
29/29 [==============================] - 0s 5ms/step - loss: 0.0260 - val_loss: 0.0290
Epoch 8/500
29/29 [==============================] - 0s 5ms/step - loss: 0.0259 - val_loss: 0.0290
Epoch 9/500
29/29 [==============================] - 0s 5ms/step - loss: 0.0256 - val_loss: 0.0289
Epoch 10/500
29/29 [==============================] - 0s 4ms/step - loss: 0.0253 - val_loss: 0.0288
Epoch 11/

In [41]:
save_keras_model('dense_with_skip', dense_with_skip)

INFO:tensorflow:Assets written to: models/dense_with_skip/assets


In [42]:
pred = dense_with_skip.predict(x_test)
validation_metrics(pred, y_test)

64.39283322986148

In [43]:
!zip -r models_NN_for_face_extraction.zip models

  adding: models/ (stored 0%)
  adding: models/dense_with_skip/ (stored 0%)
  adding: models/dense_with_skip/saved_model.pb (deflated 90%)
  adding: models/dense_with_skip/assets/ (stored 0%)
  adding: models/dense_with_skip/variables/ (stored 0%)
  adding: models/dense_with_skip/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: models/dense_with_skip/variables/variables.index (deflated 71%)
  adding: models/dense_with_skip/keras_metadata.pb (deflated 92%)
  adding: models/simple_dense/ (stored 0%)
  adding: models/simple_dense/saved_model.pb (deflated 88%)
  adding: models/simple_dense/assets/ (stored 0%)
  adding: models/simple_dense/variables/ (stored 0%)
  adding: models/simple_dense/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: models/simple_dense/variables/variables.index (deflated 62%)
  adding: models/simple_dense/keras_metadata.pb (deflated 87%)
